In [118]:
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import time
import requests 
import urllib.request

path = "C://1999//P-SAT//2021 봄과 여름//주제분석 1주차//chromedriver.exe"
driver = webdriver.Chrome(path)

In [119]:
data = pd.DataFrame(columns = ["상태","공고번호","축종","품종","털색","성별","중성화","특징","관할보호센터명","보호장소"])
data

,상태,공고번호,축종,품종,털색,성별,중성화,특징,관할보호센터명,보호장소


In [120]:
driver.get('https://www.animal.go.kr/front/awtis/public/publicList.do?menuNo=1000000055#')

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')
    
# 기간 설정
sdate = driver.find_element_by_xpath('//*[@id="searchSDate"]')
sdate.send_keys("002017-01-01")

edate = driver.find_element_by_xpath('//*[@id="searchEDate"]')
edate.send_keys("002017-12-31")

driver.find_element_by_css_selector('#search_button').click()

In [121]:
driver.find_element_by_css_selector('#searchList > ul > li.last > a').click()

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

last_result = soup.find('ul', {'class':'paging'}).find('li', {'class':'last'})
maxpage = last_result.find('a').getText()
maxpage

'10084'

In [122]:
page = 1
maxpage_num = int(maxpage)

while page <= maxpage_num:
    for i in range(1,51) :
        this_infos=[] #정보 받을 준비
        
        #이 url 안에 있는 searchSDate랑 searchEDate 바꿔주세요!
        url = 'https://www.animal.go.kr/front/awtis/public/publicList.do?totalCount=395992&pageSize=50&boardId=&desertionNo=&menuNo=1000000055&searchSDate=2017-01-01&searchEDate=2017-12-31&searchUprCd=&searchOrgCd=&searchCareRegNo=&searchUpKindCd=&searchKindCd=&searchState=&&page=' + str(page)
        driver.get(url)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
    
        #분양상태
        con = soup.select("#searchList > div.boardList > ul:nth-child(2) > li:nth-child(" +str(i) + ") > div.txt > dl:nth-child(7) > dd")
        con = con[0].text
        this_infos.append(con.strip())
    
        #상세보기
        xpath = '//*[@id="searchList"]/div[4]/ul[2]/li[' + str(i) + ']/div[1]/a'
    
        driver.find_element_by_xpath(xpath).click()
        source=driver.page_source
        soup=BeautifulSoup(source, 'html.parser')

        #동물정보
        for j in range(2,9):
            where = "#publicForm > div > table > tbody > tr:nth-child(" +str(j) + ") > td"
            info = soup.select(where)
            info = info[0].text
            info = info.strip()

            this_infos.append(info)
            
        #센터이름
        info = soup.select("#publicForm > div > table > tbody > tr:nth-child(16) > td")
        info = info[0].text
        info = info.strip()
        this_infos.append(info)        
    
        #보호장소
        info = soup.select("#publicForm > div > table > tbody > tr:nth-child(17) > td")
        info = info[0].text
        info = info.strip()
        this_infos.append(info) 
        
        data.loc[len(data)] = this_infos
        driver.back()
        
    page += 1
    
driver.close()

KeyboardInterrupt: 

In [123]:
data

,상태,공고번호,축종,품종,털색,성별,중성화,특징,관할보호센터명,보호장소
0,종료(안락사),경남-밀양-2018-00012,개,믹스견,"검정, 흰색",수컷,미상,머리와 몸은 까만털이고 다리에는 점 무늬처럼 털이 났음,밀양시동물보호소(구),경상남도 밀양시 부북면 감천리 615-4
1,종료(자연사),경남-밀양-2018-00011,개,믹스견,연한갈색,수컷,미상,발톱이 날카롭고 얼굴과 군데군데 상처가 많았음,밀양시동물보호소(구),경상남도 밀양시 부북면 감천리 615-4
2,종료(자연사),경북-포항-2018-00045,고양이,한국 고양이,"황,백",수컷,아니오,건강상태 매우 나쁨,영일동물플러스,경상북도 포항시 북구 흥해읍 덕장길 224 (흥해읍)
3,종료(자연사),충남-홍성-2018-00008,개,믹스견,흰갈검,암컷,아니오,흰갈검,금일동물보호센터,충청남도 홍성군 홍동면 충절로 625 (홍동면)
4,종료(안락사),충남-홍성-2018-00007,개,믹스견,흰갈검,암컷,아니오,흰갈검,금일동물보호센터,충청남도 홍성군 홍동면 충절로 625 (홍동면)
...,...,...,...,...,...,...,...,...,...,...
9435,종료(자연사),서울-동대문-2017-00347,기타축종,갈/흰,갈/흰,수컷,아니오,좌전지팔꿈치아래피부/뼈노출/근육소실.기립못함.오른귀밑상처/염증.눈곱/콧물.저체온.심...,한국동물구조관리협회,경기도 양주시 남면 감악산로 63-37
9436,종료(입양),서울-중구-2017-00145,기타축종,토끼,갈/흰,암컷,미상,오른눈 주위 탈모. 등부분 탈모. 코갈색. 사람손탐. 털 때탐,한국동물구조관리협회,경기도 양주시 남면 감악산로 63-37 (남면)
9437,종료(자연사),서울-중구-2017-00144,고양이,고양이,갈/흰,수컷,아니오,코분홍. 머리흔듬. 뒷다리 못씀. 항문 주변 부종/피하출혈. 저체온. 기력없음. 단...,한국동물구조관리협회,경기도 양주시 남면 감악산로 63-37 (남면)
9438,종료(반환),서울-종로-2017-00271,개,믹스견,갈/흰,암컷,미상,코검정. 으르렁거림/경계심함/짖음. 경계하여 칩확인불가. 단미안됨. 털상태 양호,한국동물구조관리협회,경기도 양주시 남면 감악산로 63-37 (남면)


In [124]:
# 파일 경로 바꾸고, 제목도 얻은 데이터에 해당하는 날짜로 바꿔주세요
data.to_excel(os.path.join("C:/1999/P-SAT/2021 봄과 여름/주제분석 1주차/", '20170101_20171231.xlsx'), sheet_name='sheet1', header=True,
                      startrow=0, startcol=0)